In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import statistics

In [2]:
hdb_df = pd.read_csv('data/hdb_df_min_dist.csv', low_memory=False)

In [3]:
hdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919408 entries, 0 to 919407
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   month                         919408 non-null  object 
 1   town                          919408 non-null  object 
 2   flat_type                     919408 non-null  object 
 3   block                         919408 non-null  object 
 4   street_name                   919408 non-null  object 
 5   storey_range                  919408 non-null  object 
 6   floor_area_sqm                919408 non-null  float64
 7   flat_model                    919408 non-null  object 
 8   lease_commence_date           919408 non-null  int64  
 9   resale_price                  919408 non-null  float64
 10  remaining_lease               210358 non-null  object 
 11  address                       919408 non-null  object 
 12  latitude                      919408 non-nul

In [4]:
hdb_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,address,latitude,longitude,nearest_supermarket_distance,nearest_school_distance,nearest_mrt_distance,nearest_hawkers_distance,cbd_distance
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN,309 ANG MO KIO AVE 1,1.366558,103.841624,0.000000,104.434561,781.530960,0.000000,8765.007560
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN,309 ANG MO KIO AVE 1,1.366558,103.841624,0.000000,104.434561,781.530960,0.000000,8765.007560
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN,309 ANG MO KIO AVE 1,1.366558,103.841624,0.000000,104.434561,781.530960,0.000000,8765.007560
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN,309 ANG MO KIO AVE 1,1.366558,103.841624,0.000000,104.434561,781.530960,0.000000,8765.007560
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN,216 ANG MO KIO AVE 1,1.366197,103.841505,42.113407,145.364227,800.632183,42.113407,8727.095898


# Data Cleaning

### remaining lease years calculation

In [5]:
# convert year of sale to int
hdb_df['year_of_sale'] = hdb_df['month'].str[:4].astype('int64')

In [6]:
hdb_df[hdb_df['lease_commence_date'] > hdb_df['year_of_sale']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 23271 to 634577
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   month                         51 non-null     object 
 1   town                          51 non-null     object 
 2   flat_type                     51 non-null     object 
 3   block                         51 non-null     object 
 4   street_name                   51 non-null     object 
 5   storey_range                  51 non-null     object 
 6   floor_area_sqm                51 non-null     float64
 7   flat_model                    51 non-null     object 
 8   lease_commence_date           51 non-null     int64  
 9   resale_price                  51 non-null     float64
 10  remaining_lease               0 non-null      object 
 11  address                       51 non-null     object 
 12  latitude                      51 non-null     float64
 13 

In [106]:
# drop rows were lease commencement date is later than the year of sale
hdb_df = hdb_df[hdb_df['lease_commence_date'] <= hdb_df['year_of_sale']]

In [107]:
# calculate remaining lease
hdb_df['calculated_remaining_lease'] = 99 - (hdb_df['year_of_sale'] - hdb_df['lease_commence_date'])
hdb_df['calculated_remaining_lease'] = hdb_df['calculated_remaining_lease'].astype(float)

### flat model

In [108]:
hdb_df['flat_model'].unique()

array(['IMPROVED', 'NEW GENERATION', 'MODEL A', 'STANDARD', 'SIMPLIFIED',
       'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE', 'TERRACE',
       '2-ROOM', 'IMPROVED-MAISONETTE', 'MULTI GENERATION',
       'PREMIUM APARTMENT', 'Improved', 'New Generation', 'Model A',
       'Standard', 'Apartment', 'Simplified', 'Model A-Maisonette',
       'Maisonette', 'Multi Generation', 'Adjoined flat',
       'Premium Apartment', 'Terrace', 'Improved-Maisonette',
       'Premium Maisonette', '2-room', 'Model A2', 'DBSS', 'Type S1',
       'Type S2', 'Premium Apartment Loft', '3Gen'], dtype=object)

In [109]:
flat_model_dict = {
    'MODEL A': 'Model A',
    'IMPROVED': 'Improved',
    'NEW GENERATION': 'New Generation',
    'STANDARD': 'Standard',
    'SIMPLIFIED': 'Simplified',
    'MODEL A-MAISONETTE': 'Model A-Maisonette',
    'APARTMENT': 'Apartment',
    'MAISONETTE': 'Maisonette',
    'TERRACE': 'Terrace',
    '2-ROOM': '2-Room',
    '2-room': '2-Room',
    'IMPROVED-MAISONETTE': 'Improved-Maisonette',
    'MULTI GENERATION': 'Multi Generation',
    'PREMIUM APARTMENT': 'Premium Apartment'    
}

In [110]:
hdb_df['flat_model'] = hdb_df['flat_model'].replace(flat_model_dict)

In [111]:
hdb_df['flat_model'].unique()

array(['Improved', 'New Generation', 'Model A', 'Standard', 'Simplified',
       'Model A-Maisonette', 'Apartment', 'Maisonette', 'Terrace',
       '2-Room', 'Improved-Maisonette', 'Multi Generation',
       'Premium Apartment', 'Adjoined flat', 'Premium Maisonette',
       'Model A2', 'DBSS', 'Type S1', 'Type S2', 'Premium Apartment Loft',
       '3Gen'], dtype=object)

### flat_type

In [112]:
hdb_df['flat_type'].unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI GENERATION', 'MULTI-GENERATION'], dtype=object)

In [113]:
# combine the MULTI-GENERATION and MULTI GENERATION labels into a single category
hdb_df['flat_type'] = hdb_df['flat_type'].replace('MULTI-GENERATION', 'MULTI GENERATION')

In [114]:
hdb_df['flat_type'].unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI GENERATION'], dtype=object)

### Storey range

In [115]:
hdb_df['storey_range'].unique()

array(['10 TO 12', '04 TO 06', '07 TO 09', '01 TO 03', '13 TO 15',
       '19 TO 21', '16 TO 18', '25 TO 27', '22 TO 24', '28 TO 30',
       '31 TO 33', '40 TO 42', '37 TO 39', '34 TO 36', '06 TO 10',
       '01 TO 05', '11 TO 15', '16 TO 20', '21 TO 25', '26 TO 30',
       '36 TO 40', '31 TO 35', '46 TO 48', '43 TO 45', '49 TO 51'],
      dtype=object)

In [116]:
# get median of storey range
def get_median(x):
    split_list = x.split(' TO ')
    float_list = [float(i) for i in split_list]
    median = statistics.median(float_list)
    return median

hdb_df['storey_median'] = hdb_df['storey_range'].apply(lambda x: get_median(x)).astype('int64')

In [117]:
hdb_df['storey_median'].unique()

array([11,  5,  8,  2, 14, 20, 17, 26, 23, 29, 32, 41, 38, 35,  3, 13, 18,
       28, 33, 47, 44, 50], dtype=int64)

In [118]:
# calculate price per sqm
hdb_df['price_per_sqm'] = hdb_df['resale_price']/hdb_df['floor_area_sqm']

In [119]:
# renaming date columns
hdb_df.rename(columns={'month': 'month_of_sale'}, inplace=True)

In [120]:
# drop remaining_lease column
hdb_df.drop(columns=['remaining_lease'], inplace=True)

In [121]:
# drop duplicate rows
hdb_df = hdb_df.drop_duplicates()

In [122]:
hdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 917393 entries, 0 to 919407
Data columns (total 22 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   month_of_sale                 917393 non-null  object 
 1   town                          917393 non-null  object 
 2   flat_type                     917393 non-null  object 
 3   block                         917393 non-null  object 
 4   street_name                   917393 non-null  object 
 5   storey_range                  917393 non-null  object 
 6   floor_area_sqm                917393 non-null  float64
 7   flat_model                    917393 non-null  object 
 8   lease_commence_date           917393 non-null  int64  
 9   resale_price                  917393 non-null  float64
 10  address                       917393 non-null  object 
 11  latitude                      917393 non-null  float64
 12  longitude                     917393 non-nul

In [123]:
hdb_df.to_csv('data/hdb_combined_clean.csv', index=False)